# Opik Client Libraries and REST API Integration Demo

This notebook demonstrates how to use Opik's suite of client libraries and REST API for seamless integration into your ML workflows. Opik provides SDKs for Python, TypeScript, and Ruby (via OpenTelemetry) for comprehensive experiment tracking and monitoring.

## Overview
- Install and configure Opik client
- Basic API operations and CRUD functionality  
- Python SDK usage examples
- Logging and tracking operations
- Error handling and best practices

For detailed API and SDK references, see the [Opik Client Reference Documentation](https://docs.opik.io/).

## 1. Install and Import Opik

First, we'll install the Opik package and import the necessary modules for working with the Opik client.

In [ ]:
# Install Opik package
import subprocess
import sys

def install_package(package):
    """Install a package using pip"""
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

try:
    import opik
    print("✓ Opik already installed")
except ImportError:
    print("Installing Opik...")
    install_package("opik")
    import opik
    print("✓ Opik installed successfully")

# Import necessary modules
import os
import json
import requests
from datetime import datetime
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print(f"Opik version: {opik.__version__}")
print("✓ All imports successful")

## 2. Initialize Opik Client

Set up and configure the Opik client with proper authentication and server connection settings.

In [ ]:
# Initialize Opik client
from opik import Opik
from opik.config import OpikConfig

# Configuration options
config = {
    "api_key": os.getenv("OPIK_API_KEY", "your-api-key-here"),
    "workspace": os.getenv("OPIK_WORKSPACE", "default"),
    "url": os.getenv("OPIK_URL", "https://www.comet.com/opik/api"),
}

print("Initializing Opik client...")
print(f"Workspace: {config['workspace']}")
print(f"URL: {config['url']}")

try:
    # Initialize Opik client
    client = Opik(
        api_key=config["api_key"],
        workspace=config["workspace"],
        url=config["url"]
    )
    
    print("✓ Opik client initialized successfully")
    
    # Test connection
    try:
        # This would test the connection (adjust based on actual Opik API)
        info = client.get_workspace_info()
        print(f"✓ Connection successful - Workspace: {info.get('name', 'N/A')}")
    except Exception as e:
        print(f"⚠ Connection test failed (this may be expected in demo): {e}")
        
except Exception as e:
    print(f"✗ Failed to initialize Opik client: {e}")
    print("Note: Make sure you have valid credentials set in environment variables")

## 3. Basic API Operations

Demonstrate basic CRUD operations using the Opik REST API, including creating, reading, updating, and deleting resources.

In [ ]:
# Basic API Operations using Opik REST API
import requests
import json

# API endpoints (adjust based on actual Opik API structure)
base_url = config["url"]
headers = {
    "Authorization": f"Bearer {config['api_key']}",
    "Content-Type": "application/json"
}

def make_api_request(method, endpoint, data=None):
    """Make a request to the Opik API"""
    url = f"{base_url.rstrip('/')}/{endpoint.lstrip('/')}"
    
    try:
        if method.upper() == "GET":
            response = requests.get(url, headers=headers)
        elif method.upper() == "POST":
            response = requests.post(url, headers=headers, json=data)
        elif method.upper() == "PUT":
            response = requests.put(url, headers=headers, json=data)
        elif method.upper() == "DELETE":
            response = requests.delete(url, headers=headers)
        else:
            raise ValueError(f"Unsupported HTTP method: {method}")
            
        return response
    except requests.exceptions.RequestException as e:
        print(f"API request failed: {e}")
        return None

# Example: Create a new experiment (adjust endpoint based on actual API)
print("=== CREATE Operation ===")
experiment_data = {
    "name": "llama-gpu-experiment",
    "description": "Test experiment for LLaMA GPU optimization",
    "tags": ["llama", "gpu", "optimization"],
    "created_at": datetime.now().isoformat()
}

create_response = make_api_request("POST", "/experiments", experiment_data)
if create_response:
    print(f"Create response status: {create_response.status_code}")
    if create_response.status_code < 400:
        print("✓ Experiment created successfully")
        created_experiment = create_response.json()
        experiment_id = created_experiment.get("id", "demo-id")
    else:
        print(f"✗ Create failed: {create_response.text}")
        experiment_id = "demo-id"  # Fallback for demo
else:
    print("⚠ Create request failed (network/auth issue)")
    experiment_id = "demo-id"

print(f"Experiment ID: {experiment_id}")

# Example: Read (GET) operation
print("\n=== READ Operation ===")
get_response = make_api_request("GET", f"/experiments/{experiment_id}")
if get_response and get_response.status_code < 400:
    print("✓ Experiment retrieved successfully")
    experiment = get_response.json()
    print(f"Name: {experiment.get('name', 'N/A')}")
    print(f"Description: {experiment.get('description', 'N/A')}")
else:
    print("⚠ Read operation demo (actual API may differ)")

# Example: Update (PUT) operation
print("\n=== UPDATE Operation ===")
update_data = {
    "description": "Updated: LLaMA GPU optimization with monitoring",
    "tags": ["llama", "gpu", "optimization", "monitoring"]
}

update_response = make_api_request("PUT", f"/experiments/{experiment_id}", update_data)
if update_response and update_response.status_code < 400:
    print("✓ Experiment updated successfully")
else:
    print("⚠ Update operation demo (actual API may differ)")

# Example: Delete operation (commented out to preserve demo data)
print("\n=== DELETE Operation ===")
print("⚠ Delete operation available but not executed in demo")
# delete_response = make_api_request("DELETE", f"/experiments/{experiment_id}")
# if delete_response and delete_response.status_code < 400:
#     print("✓ Experiment deleted successfully")

print("\n✓ Basic CRUD operations demonstrated")

## 4. Python SDK Usage Examples

Show practical examples of using the Opik Python SDK for common tasks like data logging, experiment tracking, and metric collection.

In [ ]:
# Python SDK Usage Examples
from opik import track
from opik.decorators import op
import time
import random

# Example 1: Basic tracking decorator
@track(project_name="llama-gpu-optimization")
def train_model(model_name, epochs, learning_rate):
    """Simulate model training with tracking"""
    print(f"Training {model_name} for {epochs} epochs...")
    
    # Simulate training process
    for epoch in range(epochs):
        # Simulate training metrics
        loss = random.uniform(0.5, 2.0) * (0.9 ** epoch)  # Decreasing loss
        accuracy = min(0.95, 0.6 + (epoch * 0.05))  # Increasing accuracy
        
        # Log metrics for this epoch
        if hasattr(track.current_span(), 'log_metric'):
            track.current_span().log_metric("loss", loss, step=epoch)
            track.current_span().log_metric("accuracy", accuracy, step=epoch)
            track.current_span().log_metric("learning_rate", learning_rate, step=epoch)
        
        time.sleep(0.1)  # Simulate training time
    
    # Final results
    final_metrics = {
        "final_loss": loss,
        "final_accuracy": accuracy,
        "total_epochs": epochs,
        "model_name": model_name
    }
    
    print(f"Training complete! Final accuracy: {accuracy:.3f}")
    return final_metrics

# Example 2: Manual span creation
def evaluate_model(model_name, test_size):
    """Evaluate model performance"""
    with track.start_span(
        name="model_evaluation",
        input={"model_name": model_name, "test_size": test_size}
    ) as span:
        
        # Simulate evaluation
        print(f"Evaluating {model_name} on {test_size} samples...")
        
        # Simulate evaluation metrics
        precision = random.uniform(0.8, 0.95)
        recall = random.uniform(0.75, 0.92)
        f1_score = 2 * (precision * recall) / (precision + recall)
        
        evaluation_results = {
            "precision": precision,
            "recall": recall,
            "f1_score": f1_score,
            "test_size": test_size
        }
        
        # Log evaluation results
        span.set_output(evaluation_results)
        span.log_metric("precision", precision)
        span.log_metric("recall", recall)
        span.log_metric("f1_score", f1_score)
        
        print(f"Evaluation complete! F1-score: {f1_score:.3f}")
        return evaluation_results

# Example 3: GPU optimization tracking
@op(name="gpu_optimization")
def optimize_gpu_settings(gpu_memory_gb, batch_size, precision_mode):
    """Track GPU optimization experiments"""
    
    optimization_config = {
        "gpu_memory_gb": gpu_memory_gb,
        "batch_size": batch_size,
        "precision_mode": precision_mode
    }
    
    print(f"Optimizing GPU settings: {optimization_config}")
    
    # Simulate optimization process
    throughput = random.uniform(50, 200)  # tokens/second
    memory_usage = random.uniform(0.7, 0.95) * gpu_memory_gb
    power_consumption = random.uniform(150, 300)  # watts
    
    optimization_results = {
        "throughput_tokens_per_sec": throughput,
        "memory_usage_gb": memory_usage,
        "power_consumption_watts": power_consumption,
        "efficiency_score": throughput / power_consumption
    }
    
    print(f"Optimization results: {optimization_results}")
    return optimization_results

# Run examples
print("=== Running Python SDK Examples ===")

# Example 1: Train a model
print("\n1. Training model with tracking...")
training_results = train_model(
    model_name="llama-7b-gpu",
    epochs=5,
    learning_rate=0.001
)

# Example 2: Evaluate model
print("\n2. Evaluating model...")
evaluation_results = evaluate_model(
    model_name="llama-7b-gpu",
    test_size=1000
)

# Example 3: GPU optimization
print("\n3. GPU optimization...")
gpu_results = optimize_gpu_settings(
    gpu_memory_gb=24,
    batch_size=32,
    precision_mode="fp16"
)

print("\n✓ All SDK examples completed successfully")
print("Check your Opik dashboard for tracked experiments and metrics!")

## 5. Logging and Tracking Operations

Implement logging functionality to track experiments, metrics, and other relevant data using Opik's tracking capabilities.

In [ ]:
# Advanced Logging and Tracking Operations
import numpy as np
import matplotlib.pyplot as plt
import io
import base64

class LlamaGPUTracker:
    """Enhanced tracking class for LLaMA GPU experiments"""
    
    def __init__(self, project_name="llama-gpu-experiments"):
        self.project_name = project_name
        self.current_experiment = None
        
    def start_experiment(self, experiment_name, config=None):
        """Start a new experiment with configuration"""
        self.current_experiment = {
            "name": experiment_name,
            "config": config or {},
            "metrics": {},
            "artifacts": {},
            "start_time": datetime.now()
        }
        
        print(f"Started experiment: {experiment_name}")
        if config:
            print(f"Configuration: {json.dumps(config, indent=2)}")
    
    def log_metrics(self, metrics_dict, step=None):
        """Log multiple metrics at once"""
        if not self.current_experiment:
            print("Warning: No active experiment")
            return
            
        for metric_name, value in metrics_dict.items():
            if metric_name not in self.current_experiment["metrics"]:
                self.current_experiment["metrics"][metric_name] = []
            
            self.current_experiment["metrics"][metric_name].append({
                "value": value,
                "step": step,
                "timestamp": datetime.now()
            })
    
    def log_gpu_metrics(self, gpu_stats):
        """Log GPU-specific metrics"""
        gpu_metrics = {
            "gpu_memory_used": gpu_stats.get("memory_used", 0),
            "gpu_memory_total": gpu_stats.get("memory_total", 0),
            "gpu_utilization": gpu_stats.get("utilization", 0),
            "gpu_temperature": gpu_stats.get("temperature", 0),
            "power_draw": gpu_stats.get("power_draw", 0)
        }
        
        self.log_metrics(gpu_metrics)
        print(f"Logged GPU metrics: {gpu_metrics}")
    
    def log_model_metrics(self, model_stats):
        """Log model-specific metrics"""
        model_metrics = {
            "tokens_per_second": model_stats.get("throughput", 0),
            "perplexity": model_stats.get("perplexity", 0),
            "bleu_score": model_stats.get("bleu_score", 0),
            "latency_ms": model_stats.get("latency_ms", 0)
        }
        
        self.log_metrics(model_metrics)
        print(f"Logged model metrics: {model_metrics}")
    
    def log_artifact(self, name, content, artifact_type="text"):
        """Log artifacts like plots, configs, or text files"""
        if not self.current_experiment:
            print("Warning: No active experiment")
            return
            
        self.current_experiment["artifacts"][name] = {
            "content": content,
            "type": artifact_type,
            "timestamp": datetime.now()
        }
        
        print(f"Logged artifact: {name} ({artifact_type})")
    
    def create_performance_plot(self):
        """Create and log a performance visualization"""
        if not self.current_experiment or "tokens_per_second" not in self.current_experiment["metrics"]:
            print("No performance data to plot")
            return
            
        # Extract throughput data
        throughput_data = self.current_experiment["metrics"]["tokens_per_second"]
        values = [entry["value"] for entry in throughput_data]
        steps = list(range(len(values)))
        
        # Create plot
        plt.figure(figsize=(10, 6))
        plt.plot(steps, values, 'b-', linewidth=2, label='Tokens/Second')
        plt.xlabel('Step')
        plt.ylabel('Throughput (tokens/sec)')
        plt.title('Model Performance Over Time')
        plt.grid(True, alpha=0.3)
        plt.legend()
        
        # Save plot to buffer
        buffer = io.BytesIO()
        plt.savefig(buffer, format='png', dpi=150, bbox_inches='tight')
        buffer.seek(0)
        plot_data = base64.b64encode(buffer.getvalue()).decode()
        plt.close()
        
        # Log as artifact
        self.log_artifact("performance_plot", plot_data, "image")
        print("Created and logged performance plot")
    
    def end_experiment(self):
        """End the current experiment and summarize results"""
        if not self.current_experiment:
            print("No active experiment to end")
            return
            
        end_time = datetime.now()
        duration = end_time - self.current_experiment["start_time"]
        
        print(f"\n=== Experiment Summary: {self.current_experiment['name']} ===")
        print(f"Duration: {duration}")
        print(f"Metrics logged: {len(self.current_experiment['metrics'])}")
        print(f"Artifacts created: {len(self.current_experiment['artifacts'])}")
        
        # Log final summary metrics
        for metric_name, entries in self.current_experiment["metrics"].items():
            if entries:
                latest_value = entries[-1]["value"]
                print(f"Final {metric_name}: {latest_value}")
        
        self.current_experiment = None
        print("Experiment ended successfully")

# Demo: Comprehensive experiment tracking
print("=== Comprehensive Experiment Tracking Demo ===")

# Initialize tracker
tracker = LlamaGPUTracker("llama-gpu-optimization")

# Start experiment
tracker.start_experiment(
    "llama-7b-inference-optimization",
    config={
        "model_size": "7B",
        "precision": "fp16",
        "batch_size": 32,
        "max_length": 2048,
        "gpu_model": "RTX 4090"
    }
)

# Simulate training/inference loop with comprehensive logging
print("\nSimulating inference optimization...")
for step in range(10):
    # Simulate GPU metrics
    gpu_stats = {
        "memory_used": random.uniform(18, 22),  # GB
        "memory_total": 24,
        "utilization": random.uniform(85, 98),  # %
        "temperature": random.uniform(65, 82),  # °C
        "power_draw": random.uniform(280, 350)  # watts
    }
    
    # Simulate model performance metrics
    model_stats = {
        "throughput": random.uniform(50, 120),  # tokens/sec
        "perplexity": random.uniform(3.2, 4.8),
        "bleu_score": random.uniform(0.25, 0.45),
        "latency_ms": random.uniform(15, 35)
    }
    
    # Log all metrics
    tracker.log_gpu_metrics(gpu_stats)
    tracker.log_model_metrics(model_stats)
    
    # Log additional custom metrics
    custom_metrics = {
        "step": step,
        "efficiency_score": model_stats["throughput"] / gpu_stats["power_draw"],
        "memory_efficiency": model_stats["throughput"] / gpu_stats["memory_used"]
    }
    tracker.log_metrics(custom_metrics, step=step)
    
    time.sleep(0.2)  # Simulate processing time

# Create performance visualization
tracker.create_performance_plot()

# Log configuration file as artifact
config_content = json.dumps({
    "model_config": {
        "architecture": "llama",
        "size": "7B",
        "precision": "fp16"
    },
    "optimization_settings": {
        "batch_size": 32,
        "max_length": 2048,
        "attention_optimization": True
    }
}, indent=2)

tracker.log_artifact("model_config.json", config_content, "config")

# End experiment
tracker.end_experiment()

print("\n✓ Comprehensive logging and tracking demonstration completed!")

## 6. Error Handling and Best Practices

Demonstrate proper error handling techniques and best practices when working with the Opik client libraries and API.

In [ ]:
# Error Handling and Best Practices
import functools
import traceback
from typing import Optional, Dict, Any
import time

class OpikErrorHandler:
    """Centralized error handling for Opik operations"""
    
    def __init__(self, max_retries=3, backoff_factor=1.5):
        self.max_retries = max_retries
        self.backoff_factor = backoff_factor
        self.error_log = []
    
    def log_error(self, error, context=None):
        """Log error with context information"""
        error_entry = {
            "error": str(error),
            "type": type(error).__name__,
            "context": context or {},
            "timestamp": datetime.now(),
            "traceback": traceback.format_exc()
        }
        self.error_log.append(error_entry)
        logger.error(f"Opik operation failed: {error}")
    
    def retry_with_backoff(self, func):
        """Decorator for retrying operations with exponential backoff"""
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            last_exception = None
            
            for attempt in range(self.max_retries):
                try:
                    return func(*args, **kwargs)
                except Exception as e:
                    last_exception = e
                    wait_time = self.backoff_factor ** attempt
                    
                    self.log_error(e, {
                        "function": func.__name__,
                        "attempt": attempt + 1,
                        "max_retries": self.max_retries
                    })
                    
                    if attempt < self.max_retries - 1:
                        print(f"Attempt {attempt + 1} failed, retrying in {wait_time:.1f}s...")
                        time.sleep(wait_time)
                    else:
                        print(f"All {self.max_retries} attempts failed")
            
            raise last_exception
        return wrapper

# Initialize error handler
error_handler = OpikErrorHandler()

@error_handler.retry_with_backoff
def robust_api_call(endpoint, data=None, method="GET"):
    """Example of a robust API call with error handling"""
    
    # Validate inputs
    if not endpoint:
        raise ValueError("Endpoint cannot be empty")
    
    if method not in ["GET", "POST", "PUT", "DELETE"]:
        raise ValueError(f"Unsupported HTTP method: {method}")
    
    # Simulate API call that might fail
    if random.random() < 0.3:  # 30% chance of failure for demo
        raise requests.exceptions.ConnectionError("Simulated network error")
    
    if random.random() < 0.2:  # 20% chance of auth error for demo
        raise requests.exceptions.HTTPError("401 Unauthorized")
    
    # Simulate successful response
    return {
        "status": "success",
        "endpoint": endpoint,
        "method": method,
        "data": data
    }

@error_handler.retry_with_backoff
def robust_metric_logging(metric_name, value, experiment_id=None):
    """Robust metric logging with validation and error handling"""
    
    # Input validation
    if not metric_name or not isinstance(metric_name, str):
        raise ValueError("Metric name must be a non-empty string")
    
    if not isinstance(value, (int, float)) or np.isnan(value) or np.isinf(value):
        raise ValueError(f"Invalid metric value: {value}")
    
    # Simulate logging operation
    if random.random() < 0.15:  # 15% chance of failure
        raise Exception("Failed to log metric to Opik server")
    
    print(f"✓ Logged metric: {metric_name} = {value}")
    return True

def safe_experiment_context(experiment_name, config=None):
    """Context manager for safe experiment execution"""
    experiment_started = False
    
    try:
        # Start experiment
        print(f"Starting experiment: {experiment_name}")
        experiment_started = True
        
        # Yield control to user code
        yield {
            "experiment_name": experiment_name,
            "config": config or {},
            "start_time": datetime.now()
        }
        
    except Exception as e:
        error_handler.log_error(e, {
            "experiment_name": experiment_name,
            "config": config
        })
        print(f"✗ Experiment failed: {e}")
        raise
    
    finally:
        if experiment_started:
            print(f"Cleaning up experiment: {experiment_name}")
            # Perform cleanup operations here

def validate_opik_config(config):
    """Validate Opik configuration"""
    required_fields = ["api_key", "workspace"]
    errors = []
    
    for field in required_fields:
        if not config.get(field):
            errors.append(f"Missing required field: {field}")
    
    if config.get("api_key") == "your-api-key-here":
        errors.append("Please set a valid API key")
    
    if errors:
        raise ValueError(f"Configuration errors: {', '.join(errors)}")
    
    return True

# Best Practices Examples
print("=== Error Handling and Best Practices Demo ===")

# 1. Configuration validation
print("\n1. Configuration Validation:")
try:
    test_config = {
        "api_key": "your-api-key-here",  # Invalid placeholder
        "workspace": "test-workspace"
    }
    validate_opik_config(test_config)
except ValueError as e:
    print(f"✓ Configuration validation caught error: {e}")

# Valid configuration
valid_config = {
    "api_key": "valid-test-key-123",
    "workspace": "test-workspace",
    "url": "https://api.opik.io"
}
validate_opik_config(valid_config)
print("✓ Configuration validation passed")

# 2. Robust API calls with retry
print("\n2. Robust API Calls with Retry:")
try:
    result = robust_api_call("/experiments", {"name": "test"}, "POST")
    print(f"✓ API call succeeded: {result}")
except Exception as e:
    print(f"✗ API call failed after all retries: {e}")

# 3. Safe metric logging
print("\n3. Safe Metric Logging:")
test_metrics = [
    ("valid_metric", 0.85),
    ("invalid_metric", float('nan')),  # Should fail validation
    ("another_valid", 42.0)
]

for metric_name, value in test_metrics:
    try:
        robust_metric_logging(metric_name, value)
    except ValueError as e:
        print(f"✓ Validation caught invalid metric: {e}")
    except Exception as e:
        print(f"✗ Unexpected error: {e}")

# 4. Safe experiment context
print("\n4. Safe Experiment Context:")
try:
    with safe_experiment_context("error-handling-demo") as experiment:
        print(f"Running experiment: {experiment['experiment_name']}")
        
        # Simulate some operations
        for i in range(3):
            robust_metric_logging(f"step_{i}", random.uniform(0, 1))
        
        # Simulate an error (uncomment to test error handling)
        # raise RuntimeError("Simulated experiment error")
        
        print("✓ Experiment completed successfully")
        
except Exception as e:
    print(f"Experiment handled error gracefully: {e}")

# 5. Error summary and recommendations
print("\n=== Error Summary and Recommendations ===")
print(f"Total errors logged: {len(error_handler.error_log)}")

if error_handler.error_log:
    print("\nError types encountered:")
    error_types = {}
    for error in error_handler.error_log:
        error_type = error["type"]
        error_types[error_type] = error_types.get(error_type, 0) + 1
    
    for error_type, count in error_types.items():
        print(f"  {error_type}: {count} occurrences")

print("\n=== Best Practices Summary ===")
print("✓ Always validate configuration before initializing client")
print("✓ Implement retry logic with exponential backoff")
print("✓ Validate metric values before logging")
print("✓ Use context managers for experiment lifecycle")
print("✓ Log errors with context for debugging")
print("✓ Implement graceful degradation for non-critical operations")
print("✓ Monitor API rate limits and implement backoff")
print("✓ Use environment variables for sensitive configuration")

print("\n✓ Error handling and best practices demonstration completed!")

## Conclusion and Next Steps

This notebook demonstrated comprehensive integration with Opik's client libraries and REST API for seamless ML workflow integration. 

### Key Features Covered:
- ✅ Package installation and client initialization
- ✅ Basic CRUD operations via REST API
- ✅ Python SDK decorators and tracking
- ✅ Advanced logging and metrics collection
- ✅ Error handling and best practices
- ✅ GPU optimization tracking
- ✅ Experiment lifecycle management

### Next Steps:
1. **Set up your Opik account** and get real API credentials
2. **Configure environment variables** for secure credential management
3. **Integrate with your LLaMA GPU project** using the patterns shown
4. **Explore TypeScript/Ruby SDKs** for multi-language support
5. **Review the [Opik Documentation](https://docs.opik.io/)** for advanced features

### Integration with LLaMA GPU Project:
- Use tracking decorators in your training/inference code
- Log GPU metrics during model optimization
- Track experiment configurations and results
- Monitor model performance across different hardware setups
- Create visualizations for performance analysis

For production use, remember to:
- Set proper API keys and workspace configurations
- Implement proper error handling and retry logic
- Monitor API usage and rate limits
- Use environment variables for configuration
- Set up proper logging and monitoring